In [ ]:
import textdistance
import re
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from progress.bar import IncrementalBar
from tqdm import tqdm
from string import punctuation

# **Задание 1. Дополнительное ранжирование по вероятности**

Необходимые переменные

In [ ]:
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))
id2word = {i:word for i, word in enumerate(vocab)}
vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_12693/3043281271.py:2: SyntaxWarning: invalid escape sequence '\w'
  vocab = Counter(re.findall('\w+', corpus.lower()))


Изначально предложенные функции + модифицированный get_closest_hybrid_match

In [ ]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]
    return [(id2word[top], similarities[top]) for top in topn]



def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)
    return similarities.most_common(topn)



def get_closest_hybrid_match(text, X, vec, topn=20, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]

    closest = dict(get_closest_match_with_metric(text, lookup, topn, metric=metric))

    distance = Counter()
    for word in closest.keys():
      distance[word] = metric.distance(text, word)
    probable = Counter()
    for word in closest.keys():
      probable[word] = P(word)

    merged = sorted([[k, distance[k], probable[k], closest[k]] for k in distance],
                  key=lambda x: x[1])
    merged_union = [x for xs in merged for x in xs]

    fin = {}
    for i in range(len(merged)):
      if merged_union[1::4].count(merged[i][1]) > 1:
        indices = []
        for b in range(len(merged_union[1::4])):
          if merged_union[1::4][b] == merged[i][1]:
            indices.append(b)
        same_levenst_prob = [merged[k][2] for k in indices]
        if max(same_levenst_prob) == merged[i][2]:
          fin[merged_union[::4][i]] = {'Similarity': merged[i][3],
                         'Probability': merged[i][2],
                         'Distance': merged[i][1]}
      else:
        fin[merged_union[::4][i]] = {'Similarity': merged[i][3],
                         'Probability': merged[i][2],
                         'Distance': merged[i][1]}


    return fin



N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N




def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [ ]:
get_closest_hybrid_match('сонце', X, vec)

{'конце': {'Similarity': 0.8,
  'Probability': 0.00037068798798280367,
  'Distance': 1},
 'сон': {'Similarity': 0.6,
  'Probability': 1.416024234575859e-05,
  'Distance': 2},
 'эстонцев': {'Similarity': 0.625,
  'Probability': 7.759036901785528e-07,
  'Distance': 3}}

Может быть, я что-то не так сделала, но в целом, если добавлять вот это ранжирование по P, оно начинает учитывать встречаемость по нашему корпусу. Имхо, лучше ранжировать варианты по Similarity. Или же я зря учитывала расстояние Левенштейна как оно есть (я поняла по заданию, что нужно учитывать конкретно редакционное расстояние, а не normalized_similarity.


# **Задание 2. Symspell**

* Словарь верных слов взят из файла correct_sents.
* Учитываются только удаления с расстоянием Левенштейна 1.

In [ ]:
# необходимые переменные

corpus_cs = open('correct_sents.txt', encoding='utf8').read()
vocab_correct_sents = re.findall('\w+', corpus_cs.lower())
correct_words = dict(zip(vocab_correct_sents, [P(x) for x in vocab_correct_sents]))

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_12693/4052610685.py:4: SyntaxWarning: invalid escape sequence '\w'
  vocab_correct_sents = re.findall('\w+', corpus_cs.lower())


In [ ]:
# необходимые мне функции

def delete_one_symbol(word):
    if len(word) == 1:
        return [word]
    else:
        return list(set([word.replace(i, '', 1) for i in word]))

deletion_dict = {}
for word in correct_words.keys():
    w = delete_one_symbol(word)
    for i in w:
        if i in deletion_dict.keys():
            deletion_dict[i].append(word)
        else:
            deletion_dict[i]=[word]

def mist_del(error_word):
    all_var = delete_one_symbol(error_word)
    return [i for i in all_var if i in deletion_dict.keys()]

def most_possible_variant(error_word):
    try:
        mist_help = mist_del(error_word)
        if len(mist_help) == 0:
            return deletion_dict[mist_help[0]]
        else:
            list_of_words = []
            for i in mist_help:
                list_of_words += deletion_dict[i]
                list_of_words = list(set(list_of_words))

            all_possible = {}
            for i in list_of_words:
                all_possible[i] = correct_words[i]

        return [(m, v) for m, v in {k: v for k, v in sorted(all_possible.items(),
                                    key=lambda item: item[1],
                                    reverse=True)}.items()]

    except:
        return error_word

In [ ]:
# проверка
print(most_possible_variant('гож'))

[('его', 0.0033218376735769297), ('год', 0.0008909314122475234), ('гол', 5.9550608221203935e-05), ('ого', 1.5518073803571057e-06)]


## Тестирование

In [ ]:
# необходимые переменные для тестирования

bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

In [ ]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab_correct_sents):
            pred = cashed.get(pair[1], most_possible_variant(pair[1])[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]


        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 915/915 [00:00<00:00, 2963.93it/s]


In [ ]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8892446223111555
0.21195652173913043
0.01056621109452165


### Ну и раз уж я это <del>нагуглила</del> написала...

In [ ]:
%%time

def all_variants(word, ret=['']):
  if len(word) == 0:
    return ret
  head, tail = word[0], word[1:]
  ret = ret + list(map(lambda x: x+head, ret))
  return list(set(all_variants(tail, ret)))

CPU times: user 5 μs, sys: 0 ns, total: 5 μs
Wall time: 8.82 μs


In [ ]:
print(f'Это действительно все возможные варианты удалений для слова "лингвистика" (а их аж {len(all_variants('лингвистика'))-1}): ',
       *sorted(all_variants('лингвистика')[1:]), sep='\n')

Это действительно все возможные варианты удалений для слова "лингвистика" (а их аж 1855): 
а
в
ва
ви
виа
вии
вииа
виик
виика
вик
вика
вис
виса
виси
висиа
висик
висика
виск
виска
вист
виста
висти
вистиа
вистик
вистика
вистк
вистка
вит
вита
вити
витиа
витик
витика
витк
витка
вк
вка
вс
вса
вси
всиа
всик
всика
вск
вска
вст
вста
всти
встиа
встик
встика
встк
встка
вт
вта
вти
втиа
втик
втика
втк
втка
г
га
гв
гва
гви
гвиа
гвии
гвииа
гвиик
гвиика
гвик
гвика
гвис
гвиса
гвиси
гвисиа
гвисик
гвисика
гвиск
гвиска
гвист
гвиста
гвисти
гвистиа
гвистик
гвистика
гвистк
гвистка
гвит
гвита
гвити
гвитиа
гвитик
гвитика
гвитк
гвитка
гвк
гвка
гвс
гвса
гвси
гвсиа
гвсик
гвсика
гвск
гвска
гвст
гвста
гвсти
гвстиа
гвстик
гвстика
гвстк
гвстка
гвт
гвта
гвти
гвтиа
гвтик
гвтика
гвтк
гвтка
ги
гиа
гии
гииа
гиик
гиика
гик
гика
гис
гиса
гиси
гисиа
гисик
гисика
гиск
гиска
гист
гиста
гисти
гистиа
гистик
гистика
гистк
гистка
гит
гита
гити
гитиа
гитик
гитика
гитк
гитка
гк
гка
гс
гса
гси
гсиа
гсик
гсика
гск
гска
гст
гста
гсти
г